<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #8: Modelado de temas
`Fecha de entrega: 🎃 Octubre 31, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [10]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [11]:
# Cargar datos
path = 'reviews_vidjew_es.csv'
data = pd.read_csv(path)
print(f"Tenemos {data.shape[0]:,d} documentos") #esto se hace por buena práctica, saber cuántos documentos tenemos
data['review_body'][0] #esto lo hacemos para que nos muestre un texto





Tenemos 1,000 documentos


'Buen. Buena calidad, y buena presentación.'

In [12]:
data.head() #aquí visualizamos el archivo

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [13]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ",texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

data['preprocesado'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,preprocesado
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


En la columna preprocesado vemos el texto en minúscula, sin espacios. En este caso se debe usar en la función del preprocesado, una que permita una lista de palabras y no una cadena de texto, por eso suprimimos el join.

###  `[13 pts]` Punto 2: Modelo de LDA

In [14]:
#diccionario
dictionary = Dictionary(data['preprocesado'].values)
dictionary


In [15]:
# Filtrar palabras muy frecuentes o infrecuentes

dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in data['preprocesado'].values]
corpus

[[(0, 1), (1, 2), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(5, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)],
 [(10, 1),
  (13, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1)],
 [(27, 1), (28, 1)],
 [(2, 1), (3, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)],
 [(7, 3),
  (21, 1),
  (29, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 2),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 2),
  (53, 1),
  (54, 1)],
 [(2, 1),
  (13, 1),
  (24, 1),
  (31, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1)],
 [(6, 1),
  (24, 1),
  (26, 1),
  (51, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1)],
 [(67, 1), (68, 1)],
 [(31, 1), (47, 1), (52, 1), (69, 1), (70, 1), (71, 1), (72, 1)],
 [(73, 1), (74, 1), (75, 1)],
 [(2,

In [16]:
#Train the topic model
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=7, passes=50)
model

model.print_topics(num_words=15)

[(0,
  '0.032*"llego" + 0.032*"plata" + 0.031*"bastante" + 0.026*"bien" + 0.020*"producto" + 0.018*"pulsera" + 0.016*"tiempo" + 0.016*"pone" + 0.016*"nunca" + 0.015*"regalo" + 0.014*"si" + 0.013*"dos" + 0.012*"mas" + 0.012*"pequeña" + 0.011*"bonito"'),
 (1,
  '0.037*"perfecto" + 0.035*"mando" + 0.027*"si" + 0.021*"juego" + 0.017*"funciona" + 0.017*"jugar" + 0.017*"bien" + 0.016*"producto" + 0.014*"perfectamente" + 0.011*"switch" + 0.011*"problema" + 0.010*"quedan" + 0.010*"compra" + 0.010*"original" + 0.010*"botones"'),
 (2,
  '0.081*"calidad" + 0.056*"precio" + 0.040*"buena" + 0.034*"bien" + 0.019*"regalo" + 0.018*"buen" + 0.017*"parece" + 0.017*"queda" + 0.016*"bonita" + 0.015*"bonito" + 0.015*"cierre" + 0.014*"aunque" + 0.013*"pulsera" + 0.012*"mala" + 0.012*"foto"'),
 (3,
  '0.066*"juego" + 0.024*"gusto" + 0.020*"hijo" + 0.016*"mano" + 0.016*"pena" + 0.016*"llegó" + 0.015*"todas" + 0.014*"ser" + 0.013*"regalo" + 0.013*"vale" + 0.013*"bastante" + 0.012*"encantado" + 0.012*"bonito" +

###  `[25 pts]` Punto 3: Visualización de LDA

In [17]:
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)
# pyLDAvis.save_html(lda_display, 'lda.html')

El grupo 1 tiende a asociar adjetivos positivos en relación a los sujetos o sustantivos que describen. Tenemos superlativos como "totalmente", "claramente" que refuerzan el adjetivo positivo.

El grupo 2 asocia los textos por temas de envíos, así que encontraremos léxico que hace referencia desde la tienda de despacho hasta el momento del recibo del producto.

El grupo 3 asocia los textos por la descripción de las joyas (en su mayoría). Por el grupo de palabras las descripciones son positivas.

El grupo 4 asocia los textos por temas relacionados con los envíos. Sin emabrgo, a diferencia del grupo 2, aquí las calificaciones son negativas.

La asociación de los grupos 5 y 6 es dificil de comprender. No puedo establecer una única temática ni sujetos. Consideraría que aquí se agrupan aquellas palabras distantes en la descripción de los productos.

El grupo 7 también es confuso; sin emabrgo, parece modelar temas en relación con el uso que se le da a los productos. 

